# Index Repr Demo

This notebook demonstrates the `__repr__` and `_repr_html_` methods for the custom indexes in this package.

Both `DimensionInterval` and `NDIndex` provide informative representations that show:
- The structure of the index
- Dimensions and coordinates being managed
- Value ranges and other metadata

The HTML representations are automatically used in Jupyter notebooks for rich display.

In [1]:
from IPython.display import display, HTML

from linked_indices import DimensionInterval, NDIndex
from linked_indices.example_data import multi_interval_dataset, trial_based_dataset

## DimensionInterval Repr

The `DimensionInterval` index manages multiple interval dimensions over a single continuous dimension.

### Standard repr (text)

In [2]:
# Create a dataset with DimensionInterval
ds = multi_interval_dataset()
ds = ds.drop_indexes(["time", "word", "phoneme"]).set_xindex(
    [
        "time",
        "word_intervals",
        "phoneme_intervals",
        "word",
        "part_of_speech",
        "phoneme",
    ],
    DimensionInterval,
)

# Get the index
di_index = ds.xindexes["time"]

In [3]:
# Standard text repr
print(repr(di_index))

<DimensionInterval>
  Continuous: time
    size: 1000, range: [0, 120]
  Interval dimensions:
    word:
      coord: word_intervals
      size: 3, range: [0, 120), closed: 'left'
      labels: ['word', 'part_of_speech']
    phoneme:
      coord: phoneme_intervals
      size: 6, range: [0, 120), closed: 'left'
      labels: ['phoneme']


### HTML repr (rich display)

The HTML representation shows the same information in a styled table format:

In [4]:
# HTML repr - automatically used in Jupyter
display(HTML(di_index._repr_html_()))

### DimensionInterval with onset/duration format

When intervals are constructed from onset/duration coordinates, the repr indicates this:

In [5]:
import numpy as np
import xarray as xr

# Create dataset with onset/duration format
N = 1000
times = np.linspace(0, 120, N)

ds_onset = xr.Dataset(
    {"data": (("C", "time"), np.random.rand(2, N))},
    coords={
        "time": times,
        "word_onset": ("word", [0.0, 40.0, 80.0]),
        "word_duration": ("word", [35.5, 35.5, 35.5]),
        "word": ("word", ["hello", "world", "test"]),
    },
)

ds_onset = ds_onset.drop_indexes(["time", "word"]).set_xindex(
    ["time", "word_onset", "word_duration", "word"],
    DimensionInterval,
    onset_duration_coords={"word": ("word_onset", "word_duration")},
)

onset_index = ds_onset.xindexes["time"]

In [6]:
# Text repr shows "(from onset/duration)"
print(repr(onset_index))

<DimensionInterval>
  Continuous: time
    size: 1000, range: [0, 120]
  Interval dimensions:
    word: (from onset/duration)
      size: 3, range: [0, 115.5), closed: 'left'
      labels: ['word_onset', 'word_duration', 'word']


In [7]:
# HTML repr
display(HTML(onset_index._repr_html_()))

## NDIndex Repr

The `NDIndex` manages N-dimensional derived coordinates (like `abs_time` with shape `(trial, rel_time)`).

### Standard repr (text)

In [8]:
# Create a dataset with NDIndex
ds_nd = trial_based_dataset(n_trials=3, trial_length=5.0, sample_rate=10)
ds_nd = ds_nd.set_xindex(["abs_time"], NDIndex)

# Get the index
nd_index = ds_nd.xindexes["abs_time"]

In [9]:
# Standard text repr
print(repr(nd_index))

<NDIndex>
  slice_method: 'bounding_box'
  Coordinates:
    abs_time:
      dims: (trial, rel_time)
      shape: (3 × 50)
      range: [0, 14.9]


### HTML repr (rich display)

In [10]:
# HTML repr
display(HTML(nd_index._repr_html_()))

### NDIndex with trim_outer slice method

The slice_method configuration is shown in the repr:

In [11]:
# Create with trim_outer slice method
ds_trim = trial_based_dataset(n_trials=3, trial_length=5.0, sample_rate=10)
ds_trim = ds_trim.set_xindex(["abs_time"], NDIndex, slice_method="trim_outer")

trim_index = ds_trim.xindexes["abs_time"]
print(repr(trim_index))

<NDIndex>
  slice_method: 'trim_outer'
  Coordinates:
    abs_time:
      dims: (trial, rel_time)
      shape: (3 × 50)
      range: [0, 14.9]


In [12]:
display(HTML(trim_index._repr_html_()))

### NDIndex with multiple coordinates

When multiple N-D coordinates are managed by the same index, all are shown:

In [13]:
# Create with multiple 2D coordinates
ds_multi = trial_based_dataset(n_trials=3, trial_length=5.0, sample_rate=10)
ds_multi = ds_multi.assign_coords(
    {"normalized_time": ds_multi.abs_time / float(ds_multi.abs_time.max())}
)
ds_multi = ds_multi.set_xindex(["abs_time", "normalized_time"], NDIndex)

multi_index = ds_multi.xindexes["abs_time"]
print(repr(multi_index))

<NDIndex>
  slice_method: 'bounding_box'
  Coordinates:
    abs_time:
      dims: (trial, rel_time)
      shape: (3 × 50)
      range: [0, 14.9]
    normalized_time:
      dims: (trial, rel_time)
      shape: (3 × 50)
      range: [0, 1]


In [14]:
display(HTML(multi_index._repr_html_()))

### NDIndex with 3D coordinates

NDIndex works with coordinates of any dimensionality >= 2:

In [15]:
# Create 3D dataset: subject × trial × rel_time
n_subjects, n_trials, n_times = 2, 3, 50

subjects = ["alice", "bob"]
trials = ["trial_0", "trial_1", "trial_2"]
rel_time = np.linspace(0, 5, n_times)

# Subject session offsets
subject_offset = xr.DataArray(
    [0.0, 100.0], dims=["subject"], coords={"subject": subjects}
)

# Trial onsets
trial_onset = xr.DataArray([0.0, 10.0, 20.0], dims=["trial"], coords={"trial": trials})

# 3D absolute time
abs_time_3d = subject_offset + trial_onset + xr.DataArray(rel_time, dims=["rel_time"])

ds_3d = xr.Dataset(
    {
        "signal": (
            ("subject", "trial", "rel_time"),
            np.random.randn(n_subjects, n_trials, n_times),
        )
    },
    coords={
        "subject": subjects,
        "trial": trials,
        "rel_time": rel_time,
        "abs_time": abs_time_3d,
    },
)

ds_3d = ds_3d.set_xindex(["abs_time"], NDIndex)
index_3d = ds_3d.xindexes["abs_time"]
print(repr(index_3d))

<NDIndex>
  slice_method: 'bounding_box'
  Coordinates:
    abs_time:
      dims: (subject, trial, rel_time)
      shape: (2 × 3 × 50)
      range: [0, 125]


In [16]:
display(HTML(index_3d._repr_html_()))

## Repr after slicing

The repr updates to reflect the current state of the index after slicing operations:

In [17]:
# Slice DimensionInterval dataset
ds_sliced = ds.sel(time=slice(20, 60))
_ = ds_sliced * 1  # Force evaluation

print("DimensionInterval after slicing time to [20, 60]:")
print(repr(ds_sliced.xindexes["time"]))

DimensionInterval after slicing time to [20, 60]:
<DimensionInterval>
  Continuous: time
    size: 333, range: [20.06, 59.94]
  Interval dimensions:
    word:
      coord: word_intervals
      size: 2, range: [0, 80), closed: 'left'
      labels: ['word', 'part_of_speech']
    phoneme:
      coord: phoneme_intervals
      size: 2, range: [20, 60), closed: 'left'
      labels: ['phoneme']


In [18]:
display(HTML(ds_sliced.xindexes["time"]._repr_html_()))

In [19]:
# Slice NDIndex dataset
ds_nd_sliced = ds_nd.isel(trial=slice(0, 2))

print("NDIndex after slicing to first 2 trials:")
print(repr(ds_nd_sliced.xindexes["abs_time"]))

NDIndex after slicing to first 2 trials:
<NDIndex>
  slice_method: 'bounding_box'
  Coordinates:
    abs_time:
      dims: (trial, rel_time)
      shape: (2 × 50)
      range: [0, 9.9]


In [20]:
display(HTML(ds_nd_sliced.xindexes["abs_time"]._repr_html_()))

## Summary

Both index classes provide informative representations:

### DimensionInterval repr shows:
- **Continuous dimension**: name, size, and value range
- **Interval dimensions**: 
  - Dimension name
  - Coordinate name (or "from onset/duration" indicator)
  - Size, range, and closed property
  - Associated label coordinates

### NDIndex repr shows:
- **Slice method**: 'bounding_box' or 'trim_outer'
- **N-D coordinates**:
  - Coordinate name
  - Dimensions tuple
  - Shape
  - Value range

The HTML representations use styled tables with color coding for easy reading in Jupyter notebooks.